In [4]:

import os
import numpy as np
import pydicom
from skimage import exposure

def load_mri_slices(dicom_folder):
    slices = []
    for file in sorted(os.listdir(dicom_folder)):
        if file.endswith(".dcm"):
            dicom_file = pydicom.dcmread(os.path.join(dicom_folder, file))
            slices.append(dicom_file.pixel_array)
    volume = np.stack(slices)
    return volume

def normalize_volume(volume):
    volume = (volume - np.min(volume)) / (np.max(volume) - np.min(volume))
    volume = exposure.equalize_hist(volume)
    return volume

dicom_folder = "shashankhegde06/AR-in-MedicalImages/DICOM to 3D/archive"
mri_volume = load_mri_slices(dicom_folder)
mri_volume = normalize_volume(mri_volume)
print(f"MRI Volume Shape: {mri_volume.shape}")

MRI Volume Shape: (100, 512, 512)


In [5]:

from skimage.measure import marching_cubes
from stl import mesh

def create_model(volume, output_path, level=0.5):
    verts, faces, normals, values = marching_cubes(volume, level=level)
    surface_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            surface_mesh.vectors[i][j] = verts[f[j], :]
    surface_mesh.save(output_path)

model_path = "model.stl"
create_model(mri_volume, model_path)
print(f"saved to {model_path}")


saved to model.stl
